In [ ]:
import pandas as pd

df = pd.read_csv("/content/sample_data/Crimes_-_2001_to_Present_20251129.csv")
df.head()


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,14038262,JJ500743,11/21/2025 12:00:00 AM,052XX N EAST RIVER RD,0265,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,APARTMENT,False,False,...,41.0,76.0,02,1116632.0,1933580.0,2025,11/28/2025 03:42:27 PM,41.974242,-87.846496,"(41.974242458, -87.846496423)"
1,14035676,JJ497470,11/21/2025 12:00:00 AM,055XX S WESTERN AVE,0460,BATTERY,SIMPLE,STREET,False,False,...,15.0,63.0,08B,1161312.0,1867880.0,2025,11/28/2025 03:42:27 PM,41.793142,-87.684022,"(41.793141605, -87.684021912)"
2,14037576,JJ500069,11/21/2025 12:00:00 AM,019XX W ERIE ST,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,RESIDENCE,False,False,...,36.0,24.0,26,1163400.0,1904359.0,2025,11/28/2025 03:42:27 PM,41.893200,-87.675341,"(41.893200189, -87.675340584)"
3,14038762,JJ500125,11/21/2025 12:00:00 AM,029XX N NORA AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,STREET,False,False,...,29.0,18.0,26,1128826.0,1918732.0,2025,11/28/2025 03:42:27 PM,41.933299,-87.801993,"(41.933298965, -87.801993102)"
4,14039267,JJ501948,11/21/2025 12:00:00 AM,006XX W MONTROSE AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,...,46.0,3.0,07,1170904.0,1929449.0,2025,11/28/2025 03:42:27 PM,41.961887,-87.647044,"(41.961887085, -87.647043786)"


In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date'].head()


/tmp/ipython-input-664701225.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


,Date
0,2025-11-21
1,2025-11-21
2,2025-11-21
3,2025-11-21
4,2025-11-21


In [ ]:
df['Date'].min(), df['Date'].max()


(Timestamp('2024-11-29 16:08:00'), Timestamp('2025-11-21 00:00:00'))

In [ ]:
df['Community Area'].isna().sum()


np.int64(1)

In [ ]:
df['Community Area'].unique()


array([76., 63., 24., 18.,  3., 66., 29., 28., 44., 43.,  8., 77., 67.,
       68., 59.,  7., 35., 25.,  6., 71., 48., 32., 42., 69., 52., 49.,
       16., 23.,  1., 19., 30., 15., 75.,  2., 11., 40., 72., 34., 57.,
       61., 26., 22., 53., 51., 73., 14., 65., 20., 27., 39., 38., 33.,
       62., 58., 50., 36., 46.,  5., 12., 21., 54., 31., 47., 55., 74.,
       56., 45., 60., 17., 41., 64., 70.,  4., 37., 10., 13.,  9., nan])

In [ ]:
df = df.dropna(subset=['Community Area'])


In [ ]:
df['Community Area'].isna().sum()


np.int64(0)

In [ ]:
cols_to_keep = [
    'Date',
    'Primary Type',
    'Description',
    'Location Description',
    'IUCR',
    'Community Area'
]

df = df[cols_to_keep]
df.head()


,Date,Primary Type,Description,Location Description,IUCR,Community Area
0,2025-11-21,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,APARTMENT,0265,76.0
1,2025-11-21,BATTERY,SIMPLE,STREET,0460,63.0
2,2025-11-21,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,RESIDENCE,5001,24.0
3,2025-11-21,OTHER OFFENSE,HARASSMENT BY TELEPHONE,STREET,2825,18.0
4,2025-11-21,MOTOR VEHICLE THEFT,AUTOMOBILE,PARKING LOT / GARAGE (NON RESIDENTIAL),0910,3.0


In [ ]:
df['Primary Type'].unique()


array(['CRIMINAL SEXUAL ASSAULT', 'BATTERY', 'OTHER OFFENSE',
       'MOTOR VEHICLE THEFT', 'ASSAULT', 'CRIMINAL DAMAGE',
       'OFFENSE INVOLVING CHILDREN', 'THEFT', 'DECEPTIVE PRACTICE',
       'WEAPONS VIOLATION', 'SEX OFFENSE', 'BURGLARY',
       'INTERFERENCE WITH PUBLIC OFFICER', 'PUBLIC PEACE VIOLATION',
       'NARCOTICS', 'ROBBERY', 'CRIMINAL TRESPASS', 'PROSTITUTION',
       'GAMBLING', 'CONCEALED CARRY LICENSE VIOLATION', 'KIDNAPPING',
       'STALKING', 'ARSON', 'OTHER NARCOTIC VIOLATION', 'HOMICIDE',
       'LIQUOR LAW VIOLATION', 'HUMAN TRAFFICKING', 'INTIMIDATION',
       'OBSCENITY', 'PUBLIC INDECENCY', 'NON-CRIMINAL'], dtype=object)

In [ ]:
import numpy as np
# Make everything uppercase just in case (it already is, but this is safe)
df['Primary Type'] = df['Primary Type'].str.upper()
df['Description'] = df['Description'].str.upper()

# Convenience variables
pt = df['Primary Type']
desc = df['Description']

conditions = [
    # 1. Homicide
    pt == 'HOMICIDE',

    # 2. Shooting (battery/assault/weapons with gun-related description)
    (
        pt.isin(['BATTERY', 'ASSAULT', 'WEAPONS VIOLATION']) &
        desc.str.contains('HANDGUN|FIREARM|GUN|RECKLESS DISCHARGE|SHOOTING', regex=True, na=False)
    ),

    # 3. Sexual Assault / Rape
    pt == 'CRIMINAL SEXUAL ASSAULT',

    # 4. Robbery
    pt == 'ROBBERY',

    # 5. Auto Burglary
    (pt == 'BURGLARY') & desc.str.contains('AUTO|VEHICLE|CAR', regex=True, na=False),

    # 6. Residential Burglary
    (pt == 'BURGLARY') & desc.str.contains('RESIDENCE|APARTMENT|HOME', regex=True, na=False),

    # 7. Auto Theft
    pt == 'MOTOR VEHICLE THEFT',
]

categories = [
    'HOMICIDE',
    'SHOOTING',
    'SEXUAL ASSAULT / RAPE',
    'ROBBERY',
    'AUTO BURGLARY',
    'RESIDENTIAL BURGLARY',
    'AUTO THEFT',
]


In [ ]:
df['Crime_Category'] = np.select(
    conditions,
    categories,
    default='OTHER'
)

df[['Primary Type', 'Description', 'Crime_Category']].head(10)


,Primary Type,Description,Crime_Category
0,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,SEXUAL ASSAULT / RAPE
1,BATTERY,SIMPLE,OTHER
2,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,OTHER
3,OTHER OFFENSE,HARASSMENT BY TELEPHONE,OTHER
4,MOTOR VEHICLE THEFT,AUTOMOBILE,AUTO THEFT
5,ASSAULT,AGGRAVATED - KNIFE / CUTTING INSTRUMENT,OTHER
6,CRIMINAL DAMAGE,TO VEHICLE,OTHER
7,BATTERY,SIMPLE,OTHER
8,OFFENSE INVOLVING CHILDREN,ENDANGERING LIFE / HEALTH OF CHILD,OTHER
9,CRIMINAL DAMAGE,TO VEHICLE,OTHER


In [ ]:
# Keep only the 7 main categories (drop OTHER)
df = df[df['Crime_Category'] != 'OTHER'].copy()

# Quick sanity check: how many rows left?
len(df), df['Crime_Category'].value_counts()



(38740,
 Crime_Category
 AUTO THEFT               16798
 SHOOTING                 10448
 ROBBERY                   6038
 AUTO BURGLARY             3210
 SEXUAL ASSAULT / RAPE     1591
 HOMICIDE                   429
 RESIDENTIAL BURGLARY       226
 Name: count, dtype: int64)

In [ ]:
crime_summary = (
    df
    .groupby(['Community Area', 'Crime_Category'])
    .size()
    .reset_index(name='Count')
    .sort_values(['Community Area', 'Crime_Category'])
)

crime_summary.head(20)


,Community Area,Crime_Category,Count
0,1.0,AUTO BURGLARY,40
1,1.0,AUTO THEFT,231
2,1.0,HOMICIDE,3
3,1.0,RESIDENTIAL BURGLARY,1
4,1.0,ROBBERY,83
5,1.0,SEXUAL ASSAULT / RAPE,30
6,1.0,SHOOTING,83
7,2.0,AUTO BURGLARY,87
8,2.0,AUTO THEFT,265
9,2.0,HOMICIDE,3


In [ ]:
crime_summary.to_csv("crime_summary_by_community.csv", index=False)
